In [1]:
from src.benchmark_algorithm import benchmark_algorithm
from src.algorithms.differential_evolution import DifferentialEvolution

benchmark_algorithm(DifferentialEvolution)

Function: sphere, Algorithm: DifferentialEvolution, Best found solution: 6.703796624533959e-26
Animating step #0
Animating step #0
Animating step #1
Animating step #2
Animating step #3
Animating step #4
Animating step #5
Animating step #6
Animating step #7
Animating step #8
Animating step #9
Animating step #10
Animating step #11
Animating step #12
Animating step #13
Animating step #14
Animating step #15
Animating step #16
Animating step #17
Animating step #18
Animating step #19
Animating step #20
Animating step #21
Animating step #22
Animating step #23
Animating step #24
Animating step #25
Animating step #26
Animating step #27
Animating step #28
Animating step #29
Animating step #30
Animating step #31
Animating step #32
Animating step #33
Animating step #34
Animating step #35
Animating step #36
Animating step #37
Animating step #38
Animating step #39
Animating step #40
Animating step #41
Animating step #42
Animating step #43
Animating step #44
Animating step #45
Animating step #46
Anim